In [11]:
import math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('energy/train.csv', encoding='cp949')
test = pd.read_csv('energy/test.csv', encoding='cp949')

In [4]:
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [5]:
def time(x):
    return int(x[-2:])
train['hour']=train['date_time'].apply(lambda x: time(x))
test['hour']=test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

In [6]:
test.interpolate(method='values', inplace=True)

In [7]:
train['date_time'] = pd.to_datetime(train['date_time'].apply(lambda x: str(x)+':00'))
test['date_time'] = pd.to_datetime(test['date_time'].apply(lambda x: str(x)+':00'))

In [8]:
#휴일여부

def holiday(x):
    if str(x.date()) == '2020-06-06':
        return 1
    elif str(x.date()) == '2020-08-15':
        return 1
    elif x.weekday() == 5 or x.weekday() == 6:
        return 1
    else: 
        return 0

train['holiday'] = train['date_time'].apply(lambda x: holiday(x))
test['holiday'] = test['date_time'].apply(lambda x: holiday(x))

In [9]:
#체감온도, 불쾌지수

def get_pow(series):
    return math.pow(series, 0.15)

train['perceived_temperature'] = 13.12 + 0.6215*train['기온(°C)'] - 11.37*train['풍속(m/s)'].apply(get_pow) + 0.3965*train['풍속(m/s)'].apply(get_pow)*train['기온(°C)']
train['discomfort_index'] = 1.8*train['기온(°C)'] - 0.55*(1-train['습도(%)']/100)*(1.8*train['기온(°C)']-26) + 32

test['perceived_temperature'] = 13.12 + 0.6215*test['기온(°C)'] - 11.37*test['풍속(m/s)'].apply(get_pow) + 0.3965*test['풍속(m/s)'].apply(get_pow)*test['기온(°C)']
test['discomfort_index'] = 1.8*test['기온(°C)'] - 0.55*(1-test['습도(%)']/100)*(1.8*test['기온(°C)']-26) + 32

In [10]:
train.to_csv('edit_train.csv', index=False)
test.to_csv('edit_test.csv', index=False)

In [7]:
# cluster_df = train['전력사용량(kWh)'].values.reshape(60, -1)

# inertia_arr = []
# k_arr = range(2, 11)

# for k in k_arr:
#     kmeans = KMeans(n_clusters=k, init='k-means++', random_state=0)
#     kmeans.fit(cluster_df)
#     interia = kmeans.inertia_
    
#     print('k: {0}, inertia : {1:.4f}'.format(k, interia))
#     inertia_arr.append(interia)

# plt.plot(k_arr, inertia_arr)
# plt.xlabel('k')
# plt.ylabel('inertia')

**2040 = 85 * 24**

일자별 클러스터

In [14]:
# #num_hour_cluster군집화, 건물별 insert

# train_cluster_df = train['전력사용량(kWh)'].values.reshape(60, -1)

# kmeans = KMeans(n_clusters=10, init='k-means++', random_state=0)
# kmeans.fit(train_cluster_df)
# clusters = kmeans.predict(train_cluster_df)

# num_hour_df['num_hour_assignment'] = clusters
# train = train.merge(num_hour_df, how='left', on='num')
# test = test.merge(num_hour_df, how='left', on='num')